In [8]:
import pickle
from sklearn.metrics import confusion_matrix
import requests, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from csv import writer
import requests, bs4
import re
%matplotlib inline
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
db = pd.read_csv('WNBA14.csv')
db = db.dropna()
db = db.reset_index(drop = True)

In [3]:
db['Opp TSP'] = np.zeros(len(db['Team']))
db['Opp TOV'] = np.zeros(len(db['Team']))
db['Opp ORB'] = np.zeros(len(db['Team']))
db['Opp FTR'] = np.zeros(len(db['Team']))
db['Opp Poss'] = np.zeros(len(db['Team']))
db['Opp OEff'] = np.zeros(len(db['Team']))
db['Opp DEff'] = np.zeros(len(db['Team']))

for i, team in enumerate(db['Team']):
    print('Team', team, 'Game Number', db['Game Num'][i])
    db['Opp TSP'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['TSP']
    db['Opp TOV'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['TOV']
    db['Opp ORB'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['ORB']
    db['Opp FTR'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['FTR']
    db['Opp Poss'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['Poss']
    db['Opp OEff'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['OEff']
    db['Opp DEff'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opp'] == team)]['DEff']

Team ATL Game Number 1


/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/ahelgeso/an

Team ATL Game Number 2
Team ATL Game Number 3
Team ATL Game Number 4
Team ATL Game Number 5
Team ATL Game Number 6
Team ATL Game Number 7
Team ATL Game Number 8
Team ATL Game Number 9
Team ATL Game Number 10
Team ATL Game Number 11
Team ATL Game Number 12
Team ATL Game Number 13
Team ATL Game Number 14
Team ATL Game Number 15
Team ATL Game Number 16
Team ATL Game Number 17
Team ATL Game Number 18
Team ATL Game Number 19
Team ATL Game Number 20
Team ATL Game Number 21
Team ATL Game Number 22
Team ATL Game Number 23
Team ATL Game Number 24
Team ATL Game Number 25
Team ATL Game Number 26
Team ATL Game Number 27
Team ATL Game Number 28
Team ATL Game Number 29
Team ATL Game Number 30
Team ATL Game Number 31
Team ATL Game Number 32
Team ATL Game Number 33
Team ATL Game Number 34
Team WAS Game Number 1
Team WAS Game Number 2
Team WAS Game Number 3
Team WAS Game Number 4
Team WAS Game Number 5
Team WAS Game Number 6
Team WAS Game Number 7
Team WAS Game Number 8
Team WAS Game Number 9
Team WAS 

In [4]:
### 4 game rolling averages

TSP4DB = pd.DataFrame()
rollingTSP = []
for team in db['Team'].unique():
    teamRollingTSP = db[db['Team'] == team]['TSP'].rolling(4).mean()
    rollingTSP.append(teamRollingTSP)
    TSP = {"Rolling4TSP":teamRollingTSP}
    TSP4df = pd.DataFrame.from_dict(TSP)
    TSP4df = TSP4df.shift(1)
    TSP4DB = pd.concat([TSP4DB, TSP4df])

TOV4DB = pd.DataFrame()
rollingTOV = []
for team in db['Team'].unique():
    teamRollingTOV = db[db['Team'] == team]['TOV'].rolling(4).mean()
    rollingTOV.append(teamRollingTOV)
    TOV = {"Rolling4TOV":teamRollingTOV}
    TOV4df = pd.DataFrame.from_dict(TOV)
    TOV4df = TOV4df.shift(1)
    TOV4DB = pd.concat([TOV4DB, TOV4df])
    
ORB4DB = pd.DataFrame()
rollingORB = []
for team in db['Team'].unique():
    teamRollingORB = db[db['Team'] == team]['ORB'].rolling(4).mean()
    rollingORB.append(teamRollingORB)
    ORB = {"Rolling4ORB":teamRollingORB}
    ORB4df = pd.DataFrame.from_dict(ORB)
    ORB4df = ORB4df.shift(1)
    ORB4DB = pd.concat([ORB4DB, ORB4df])
    
FTR4DB = pd.DataFrame()
rollingFTR = []
for team in db['Team'].unique():
    teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(4).mean()
    rollingFTR.append(teamRollingFTR)
    FTR = {"Rolling4FTR":teamRollingFTR}
    FTR4df = pd.DataFrame.from_dict(FTR)
    FTR4df = FTR4df.shift(1)
    FTR4DB = pd.concat([FTR4DB, FTR4df])
    
Poss4DB = pd.DataFrame()
rollingPoss = []
for team in db['Team'].unique():
    teamRollingPoss = db[db['Team'] == team]['Poss'].rolling(4).mean()
    rollingPoss.append(teamRollingPoss)
    POSS = {"Rolling4Poss":teamRollingPoss}
    PACE4df = pd.DataFrame.from_dict(POSS)
    PACE4df = PACE4df.shift(1)
    Poss4DB = pd.concat([Poss4DB, PACE4df])

OEff4DB = pd.DataFrame()
rollingOEff = []
for team in db['Team'].unique():
    teamRollingOEff = db[db['Team'] == team]['OEff'].rolling(4).mean()
    rollingOEff.append(teamRollingOEff)
    OEff = {"Rolling4OEff":teamRollingOEff}
    OEff4df = pd.DataFrame.from_dict(OEff)
    OEff4df = OEff4df.shift(1)
    OEff4DB = pd.concat([OEff4DB, OEff4df])
    
DEff4DB = pd.DataFrame()
rollingDEff = []
for team in db['Team'].unique():
    teamRollingDEff = db[db['Team'] == team]['DEff'].rolling(4).mean()
    rollingDEff.append(teamRollingDEff)
    DEff = {"Rolling4DEff":teamRollingDEff}
    DEff4df = pd.DataFrame.from_dict(DEff)
    DEff4df = DEff4df.shift(1)
    DEff4DB = pd.concat([DEff4DB, DEff4df])
    
db = pd.concat([db, TSP4DB, TOV4DB, ORB4DB, FTR4DB, Poss4DB, OEff4DB, DEff4DB], axis =1)

In [6]:
db.to_csv('WNBA14.csv', index = False)

In [46]:
db = pd.read_csv('WNBA14.csv')

In [5]:
db['Opp R4TSP'] = np.zeros(len(db['Team']))
db['Opp R4TOV'] = np.zeros(len(db['Team']))
db['Opp R4ORB'] = np.zeros(len(db['Team']))
db['Opp R4FTR'] = np.zeros(len(db['Team']))
db['Opp R4Poss'] = np.zeros(len(db['Team']))
db['Opp R4OEff'] = np.zeros(len(db['Team']))
db['Opp R4DEff'] = np.zeros(len(db['Team']))

for i, opp in enumerate(db['Opp']):
    db['Opp R4TSP'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4TSP']
    db['Opp R4TOV'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4TOV']
    db['Opp R4ORB'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4ORB']
    db['Opp R4FTR'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4FTR']
    db['Opp R4Poss'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4Poss']
    db['Opp R4OEff'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4OEff']
    db['Opp R4DEff'][i] = db[(db['Team'] == opp) & (db['Game Num'] == db['Game Num'][i])]['Rolling4DEff']

/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if s

In [17]:
db = pd.read_csv('WNBA16.csv')

In [18]:
ImpliedProb = []
for n in list(db['Odds']):
    if n < 0:
        ImpliedProb.append(abs(n)/((abs(n))+100))
    elif n > 0:
        ImpliedProb.append(100/(n+100))
    else:
        ImpliedProb.append(np.nan)

In [19]:
db = pd.concat([db, pd.DataFrame(ImpliedProb)], axis = 1)
db.to_csv('WNBA16.csv', index = False)